<a href="https://colab.research.google.com/github/AstronautCharlie/NLP/blob/master/Copy_of_PS1_Shakespeare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this experiment, you will train models to distringuish examples of two different genres of Shakespeare's plays: comedies and tragedies. (We'll ignore the histories, sonnets, etc.) Since he died four hundred years ago, Shakespeare has not written any more plays—although scraps of various other works have come to light. We are not, therefore, interested in building models simply to help categorize an unbounded stream of future documents, as we might be in other applications of text classification; rather, we are interested in what a classifier might have to tell us about what we mean by the terms “comedy” and “tragedy”.

You will start by copying and running your `createBasicFeatures` function from the experiment with movie reviews. Do the features the classifier focuses on tell you much about comedy and tragedy in general?

You will then implement another featurization function `createInterestingFeatures`, which will focus on only those features you think are informative for distinguishing between comedy and tragedy. Accuracy on leave-one-out cross-validation may go up, but it more important to look at the features given the highest weight by the classifier. Interpretability in machine learning, of course, may be harder to define than accuracy—although accuracy at some tasks such as summarization is hard enoough.

In [0]:
import json
import requests
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate,LeaveOneOut
from sklearn.feature_extraction.text import CountVectorizer 
from nltk import word_tokenize 
import nltk 
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer 
import numpy as np

class LemmaTokenizer(object): 
  def __init__(self):
    self.wnl = WordNetLemmatizer()
  def __call__(self, data):
    return [self.wnl.lemmatize(t) for t in word_tokenize(data)]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
#read in the shakespeare corpus
def readShakespeare():
  raw = requests.get("https://raw.githubusercontent.com/mutherr/CS6120-PS1-data/master/shakespeare_plays.json").text.strip()
  corpus = [json.loads(line) for line in raw.split("\n")]

  #remove histories from the data, as we're only working with tragedies and comedies
  corpus = [entry for entry in corpus if entry["genre"] != "history"]
  return corpus

This is where you will implement two functions to featurize the data:

In [0]:
#NB: The current contents are for testing only
#This function should return: 
#  -a sparse numpy matrix of document features
#  -a list of the correct genre for each document
#  -a list of the vocabulary used by the features, such that the ith term of the
#    list is the word whose counts appear in the ith column of the matrix. 

# This function should create a feature representation using all tokens that
# contain an alphabetic character.
def createBasicFeatures(corpus):
  
  # Get texts of plays
  plays = [corpus[i]['text'] for i in range(len(corpus))]

  # Get genres of plays 
  genres = [corpus[i]['genre'] for i in range(len(corpus))]

  # Create matrix of word counts
  vectorizer = CountVectorizer(token_pattern = '[a-zA-Z]+')
  texts = vectorizer.fit_transform(plays)

  # Get vocab 
  vocab = vectorizer.get_feature_names()

  return texts,genres,vocab

# This function can add other features you want that help classification
# accuracy, such as bigrams, word prefixes and suffixes, etc.
def createInterestingFeatures(corpus):
  # Get texts of plays 
  plays = [corpus[i]['text'] for i in range(len(corpus))]

  # Get genres of plays 
  genres = [corpus[i]['genre'] for i in range(len(corpus))]

  # Get matrix of n-gram counts
  # Use bigrams and exclude tokens with less than 0.2 frequency to exclude character names 
  # as much as possible 
  vectorizer = CountVectorizer(token_pattern='[a-zA-Z]+',
                               ngram_range=(1,2),
                               stop_words='english',
                               min_df=0.3,
                               max_df=0.8
                               )  
  texts = vectorizer.fit_transform(plays)

  # Get vocab 
  vocab = vectorizer.get_feature_names() 

  return texts,genres,vocab

In [0]:
#given a numpy matrix representation of the features for the training set, the 
# vector of true classes for each example, and the vocabulary as described 
# above, this computes the accuracy of the model using leave one out cross 
# validation and reports the most indicative features for each class
def evaluateModel(X,y,vocab,penalty="l1"):
  #create and fit the model
  model = LogisticRegression(penalty=penalty,solver="liblinear")
  results = cross_validate(model,X,y,cv=LeaveOneOut())
  
  #determine the average accuracy
  scores = results["test_score"]
  avg_score = sum(scores)/len(scores)
  
  #determine the most informative features
  # this requires us to fit the model to everything, because we need a
  # single model to draw coefficients from, rather than 26
  model.fit(X,y)
  neg_class_prob_sorted = model.coef_[0, :].argsort()
  pos_class_prob_sorted = (-model.coef_[0, :]).argsort()

  termsToTake = 20
  pos_indicators = [vocab[i] for i in neg_class_prob_sorted[:termsToTake]]
  neg_indicators = [vocab[i] for i in pos_class_prob_sorted[:termsToTake]]

  return avg_score,pos_indicators,neg_indicators

def runEvaluation(X,y,vocab):
  print("----------L1 Norm-----------")
  avg_score,pos_indicators,neg_indicators = evaluateModel(X,y,vocab,"l1")
  print("The model's average accuracy is %f"%avg_score)
  print("The most informative terms for pos are: %s"%pos_indicators)
  print("The most informative terms for neg are: %s"%neg_indicators)
  #this call will fit a model with L2 normalization
  print("----------L2 Norm-----------")
  avg_score,pos_indicators,neg_indicators = evaluateModel(X,y,vocab,"l2")
  print("The model's average accuracy is %f"%avg_score)
  print("The most informative terms for pos are: %s"%pos_indicators)
  print("The most informative terms for neg are: %s"%neg_indicators)
  

In [0]:
corpus = readShakespeare()

Run the following to train and evaluate two models with basic features:

In [79]:
X,y,vocab = createBasicFeatures(corpus)
runEvaluation(X, y, vocab)

----------L1 Norm-----------
The model's average accuracy is 0.653846
The most informative terms for pos are: ['you', 'duke', 'helena', 'prospero', 'i', 'sir', 'leontes', 'a', 'private', 'preserving', 'preservers', 'preserver', 'preserved', 'preserve', 'preservative', 'president', 'preservation', 'presents', 'presentment', 'presently']
The most informative terms for neg are: ['him', 's', 'iago', 'imogen', 'brutus', 'lear', 'o', 'rom', 'ham', 'and', 'the', 'preserving', 'preservers', 'preserver', 'pretense', 'preserved', 'preserve', 'pretend', 'president', 'pretext']
----------L2 Norm-----------
The model's average accuracy is 0.769231
The most informative terms for pos are: ['i', 'you', 'duke', 'prospero', 'a', 'helena', 'your', 'antonio', 'sir', 'leontes', 'hermia', 'for', 'lysander', 'ariel', 'sebastian', 'demetrius', 'camillo', 'stephano', 'me', 'parolles']
The most informative terms for neg are: ['iago', 'othello', 's', 'him', 'imogen', 'what', 'lear', 'brutus', 'his', 'cassio', 'o

Run the following to train and evaluate two models with features that are interesting for distinguishing comedy and tragedy:

In [73]:
X,y,vocab = createInterestingFeatures(corpus)
runEvaluation(X, y, vocab)

----------L1 Norm-----------
The model's average accuracy is 0.807692
The most informative terms for pos are: ['fairy', 'signior', 'clown', 'duke', 'antonio', 'princess', 'shepherd', 'page', 'chain', 'raging', 'push', 'putting', 'qualities', 'quality', 'quarrel', 'rais d', 'quarrels', 'quarter', 'quench', 'rags']
The most informative terms for neg are: ['general', 'corn', 'queen', 'nurse', 'poet', 'witch', 'rome', 'caesar', 'abate', 'qualities', 'quality', 'quarrel', 'quarrels', 'quarter', 'quench', 'quiet', 'quoth', 'putting', 'r', 'race']
----------L2 Norm-----------
The model's average accuracy is 0.846154
The most informative terms for pos are: ['antonio', 'duke', 'shepherd', 'clown', 'signior', 'widow', 'count', 'chain', 'fairy', 'princess', 'lion', 'prince', 'wall', 'jest', 'page', 'wood', 'monster', 'woo', 'worship', 'dance']
The most informative terms for neg are: ['general', 'nurse', 'witch', 'queen', 'corn', 'whore', 'rome', 'poet', 'trumpet', 'good night', 'slain', 'poison',